# Зачем это нужно

Ситуация: Horns and Hooves Inc. выпустила новый продукт - SmartSpoon. После запуска
на кикстартере оказалось, что людям понравилась концепция и она активно обсуждается
в соцсетях.
 - упоминаний много
 - общий тренд непонятен
 - полноценный анализ вручную стоит дорого

# Что делать

Текст можно проанализировать автоматически и определить эмоции, которые выражают
люди.
 - оценить, насколько хорошо приняли продукт
 - ответить на самые радикальные отзывы
 - определить основные жалобы

# Как сделать
 - создать или взять готовую модель распознавания
 - достать и проанализировать отзывы

# Сложнее

![](static/CloudML.png)

# Проще

![](static/sentiment1.png)

![](static/sentiment2.png)

# Песочница

[Тут](https://cloud.google.com/natural-language) можно поиграться с распознаванием
А [здесь](https://cloud.google.com/natural-language/docs/quickstarts) быстрый старт разработки с Natural Language API


**Анализ тональности текста** позволяет определить его эмоциональный окрас
и оценить мнение автора об упомянутых сущностях.

Основная цель - нахождение мнений в тексте и выявление их свойств.

Простой набор оценок:
 - позитивная
 - негативная
 - нейтральная

Напишем простую функцию для анализа тональности с помощью Google Natural Language API.

Google предоставляет оценку эмоциональности парой из двух чисел:
 - score - соответствует эмоциональному уклону текста от -1.0 (негативный) до 1.0 (позитивный)
 - magnitude - оценивает общую эмоциональность текста от 0.0 (безэмоциональный) до бесконечности

|Окрас|Значения|
|---|---|
|Позитивный|"score": 0.8, "magnitude": 3.0|
|Негативный|"score": -0.6, "magnitude": 4.0|
|Нейтральный|"score": 0.1, "magnitude": 0.0|
|Смешанный|"score": 0.0, "magnitude": 4.0|

Пример ответа API:

```json
{
  "documentSentiment": {
    "score": 0.2,
    "magnitude": 3.6
  },
  "language": "en",
   "sentences": [
    {
      "text": {
        "content": "Four score and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty and dedicated to the proposition that all men are created equal.",
        "beginOffset": 0
      },
      "sentiment": {
        "magnitude": 0.8,
        "score": 0.8
      }
    },
   ...
}
```

In [23]:
class Emotions:  # Наш класс для оценки фрагмента текста исходя из полученных score и magnitude
    POSITIVE = "😀"
    NEGATIVE = "😔"
    NEUTRAL = "😶"
    MIXED = "🥴"

    @staticmethod
    def detect(score, magnitude):  # Статический метод, возвращающий оценку
        if magnitude < 0.25:  # Не особо эмоциональный текст
            return Emotions.NEUTRAL

        if score > 0.3:
            return Emotions.POSITIVE
        elif score < -0.3:
            return Emotions.NEGATIVE
        return Emotions.MIXED

In [24]:
import os

from google.cloud import language_v1

# Указание пути к ключам от гугла, см. ссылку в разделе Песочница
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/oleh_rybalchenko/.ssh/GCP_NLP.json'

client = language_v1.LanguageServiceClient()

def analyze_sentiment(text: str, silent: bool=False) -> dict:
    """
    Функция для анализа текста
    :param text: собственно, текст
    :param silent: True для отключения печати в stdout
    :return: результаты анализа
    """

    # подготовим документ
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

    # и проанализируем его
    sentiment = client.analyze_sentiment(request={'document': document})
    results = {"sentences": []}

    # обработаем каждое предложение
    for sentence in sentiment.sentences:
        val = Emotions.detect(sentence.sentiment.score, sentence.sentiment.magnitude)
        message = f"{sentence.text.content} - {val} " \
            f"(score={sentence.sentiment.score}, magnitude={sentence.sentiment.magnitude})"
        not silent and print(message)
        results['sentences'].append(message)

    # и общую оценку
    total_val = Emotions.detect(sentiment.document_sentiment.score, sentiment.document_sentiment.magnitude)
    message = f"Overall attitude: {total_val}" \
        f"(score={sentiment.document_sentiment.score}, magnitude={sentiment.document_sentiment.magnitude})"
    not silent and print(f"\n{message}\n\n")
    results['total'] = message

    return results

Например, негативный эмоциональный окрас:

In [25]:
analyze_sentiment(
    "Modesty and conscientiousness receive their reward only in novels. "
    "In life they are exploited and then shoved aside."
)

Modesty and conscientiousness receive their reward only in novels. - 😔 (score=-0.4000000059604645, magnitude=0.4000000059604645)
In life they are exploited and then shoved aside. - 😔 (score=-0.699999988079071, magnitude=0.699999988079071)

Overall attitude: 😔(score=-0.6000000238418579, magnitude=1.2000000476837158)




Или что-нибудь повеселее:

In [2]:
analyze_sentiment(
    "Happiness is when what you think, what you say and what you do are in harmony."
)

NameError: name 'analyze_sentiment' is not defined

Расскажем анализатору шутейку:

In [27]:
analyze_sentiment(
    "Why do front end developers eat lunch alone? They don't know how to join tables."
)

Why do front end developers eat lunch alone? - 😔 (score=-0.5, magnitude=0.5)
They don't know how to join tables. - 😔 (score=-0.699999988079071, magnitude=0.699999988079071)

Overall attitude: 😔(score=-0.6000000238418579, magnitude=1.2999999523162842)




И еще:

In [28]:
analyze_sentiment(
    "I threw a boomerang a few years ago. I now live in constant fear"
)

I threw a boomerang a few years ago. - 😶 (score=-0.20000000298023224, magnitude=0.20000000298023224)
I now live in constant fear - 😔 (score=-0.5, magnitude=0.5)

Overall attitude: 😔(score=-0.30000001192092896, magnitude=0.699999988079071)




Никто не говорил, что будет просто:


In [29]:
analyze_sentiment(
    "Five stars because I do hate this awesome brand. Look at this crap!"
)

Five stars because I do hate this awesome brand. - 😀 (score=0.6000000238418579, magnitude=0.6000000238418579)
Look at this crap! - 😔 (score=-0.800000011920929, magnitude=0.800000011920929)

Overall attitude: 🥴(score=0.0, magnitude=1.5)




А теперь затянем что-нибудь из твиттера

In [1]:
from mocks import tweepy

auth = tweepy.OAuthHandler("MY_CONSUMER_KEY", "MY_CONSUMER_SECRET")
auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")

api = tweepy.API(auth)

Поищем твиты по какому-нибудь запросу и проанализируем их текст:

In [ ]:
tweets = api.search(q="nasa")

for status in tweets['statuses']:
    analyze_sentiment(status['text'])

# Бонус

Запакуем это добро в сервис

In [12]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def analyze(q: str):
    tweets = api.search(q)
    res = []
    for status in tweets['statuses']:
        res.append(
            analyze_sentiment(status['text'], silent=True)
        )
    return res

# uvicorn.run(app, port=8888)

SyntaxError: invalid syntax (<ipython-input-12-c4b08dfa46b5>, line 8)

# Пока на этом все

## Спасибо за внимание:)

Посмотреть и попробовать эту штуку можно тут:
![](static/qr.png)